In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = 'https://github.com/trending' 
response = get(url)
soup = BeautifulSoup(response.text, 'html.parser')
headings = soup.select('h1.h3.lh-condensed')
headings[0]

<h1 class="h3 lh-condensed">
<a data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"TRENDING_REPOSITORIES_PAGE","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":null,"record_id":39840932,"originating_url":"https://github.com/trending","user_id":null}}' data-hydro-click-hmac="b5c21c00a03199cc476d548acb97ab258f58b11b86fc31b42a24574f04824f67" data-view-component="true" href="/google/googletest">
<svg aria-hidden="true" class="octicon octicon-repo mr-1 color-fg-muted" data-view-component="true" height="16" version="1.1" viewbox="0 0 16 16" width="16">
<path d="M2 2.5A2.5 2.5 0 014.5 0h8.75a.75.75 0 01.75.75v12.5a.75.75 0 01-.75.75h-2.5a.75.75 0 110-1.5h1.75v-2h-8a1 1 0 00-.714 1.7.75.75 0 01-1.072 1.05A2.495 2.495 0 012 11.5v-9zm10.5-1V9h-8c-.356 0-.694.074-1 .208V2.5a1 1 0 011-1h8zM5 12.25v3.25a.25.25 0 00.4.2l1.45-1.087a.25.25 0 01.3 0L8.6 15.7a.25.25 0 00.4-.2v-3.25a.25.25 0 00-.25-.25h-3.5a.25.25 0 00-.25.25z" fill-r

In [5]:
len(headings)

25

In [6]:
[h.select_one('a').attrs['href'][1:] for h in soup.select('h1.h3.lh-condensed')]

['google/googletest',
 'projectdiscovery/nuclei-templates',
 'digitalocean/nginxconfig.io',
 'flutter/flutter',
 'PaddlePaddle/PaddleOCR',
 'flutter/pinball',
 'wolfogre/go-pprof-practice',
 'supabase/supabase',
 'felipefialho/frontend-challenges',
 'flutter/samples',
 'alibaba/fastjson2',
 'florinpop17/app-ideas',
 'charmbracelet/bubbletea',
 'huggingface/transformers',
 'databricks-academy/data-engineering-with-databricks',
 'hectorqin/reader',
 'Azure/azure-rest-api-specs',
 'terra-money/core',
 'saltstack/salt',
 'PKUFlyingPig/cs-self-learning',
 'actions/virtual-environments',
 'jojoldu/junior-recruit-scheduler',
 'dotnet/aspnetcore',
 'danielgindi/Charts',
 'microsoft/unilm']

In [ ]:
import pandas as pd
import os

#### Pulled from class will need to add values

"""
A module for obtaining repo readme and language data from the github API.
Before using this module, read through it, and follow the instructions marked
TODO.
After doing so, run it like this:
    python acquire.py
To create the `data.json` file that contains the data.
"""
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from env import github_token, github_username

# TODO: Make a github personal access token.
#     1. Go here and generate a personal access token https://github.com/settings/tokens
#        You do _not_ need select any scopes, i.e. leave all the checkboxes unchecked
#     2. Save it in your env.py file under the variable `github_token`
# TODO: Add your github username to your env.py file under the variable `github_username`
# TODO: Add more repositories to the `REPOS` list below.

REPOS = [
 
 'topkecleon/telegram-bot-bash',
 'evil-mad/EggBot',
 'telegram-bot-rb/telegram-bot',
 'alfficcadenti/splinterlands-bot',
 'nodeWechat/wechat4u',
 'tucnak/telebot',
 'CarlGroth/Carl-Bot',
 'grapeot/WechatForwardBot',
 'googleworkspace/hangouts-chat-samples',
 'GreyWolfDev/Werewolf',
 'abdelhai/awesome-bots',
 'mdgspace/bot',
 'Sank6/Discord-Bot-List',
 'samc621/SneakerBot',
 'boto/boto3-sample',
 'huseinzol05/Stock-Prediction-Models',
 'SAPConversationalAI/Webchat',
 'soumyadityac/youtube-viewer',
 'CodeWithJoe2020/pancakeswapBot',
 'Merubokkusu/discord-spam-bots',
 'ZeroDiscord/EconomyBot',
 'scrapinghub/slackbot',
 'baidu/boteye',
 'kcloze/swoole-bot',
 'hyperchessbot/hyperbot',
 'yangyuan/hearthrock',
]

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )


def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data


def get_repo_language(repo: str) -> str:
    url = f"https://api.github.com/repos/{repo}"
    repo_info = github_api_request(url)
    if type(repo_info) is dict:
        repo_info = cast(Dict, repo_info)
        if "language" not in repo_info:
            raise Exception(
                "'language' key not round in response\n{}".format(json.dumps(repo_info))
            )
        return repo_info["language"]
    raise Exception(
        f"Expecting a dictionary response from {url}, instead got {json.dumps(repo_info)}"
    )


def get_repo_contents(repo: str) -> List[Dict[str, str]]:
    url = f"https://api.github.com/repos/{repo}/contents/"
    contents = github_api_request(url)
    if type(contents) is list:
        contents = cast(List, contents)
        return contents
    raise Exception(
        f"Expecting a list response from {url}, instead got {json.dumps(contents)}"
    )


def get_readme_download_url(files: List[Dict[str, str]]) -> str:
    """
    Takes in a response from the github api that lists the files in a repo and
    returns the url that can be used to download the repo's README file.
    """
    for file in files:
        if file["name"].lower().startswith("readme"):
            return file["download_url"]
    return ""


def process_repo(repo: str) -> Dict[str, str]:
    """
    Takes a repo name like "gocodeup/codeup-setup-script" and returns a
    dictionary with the language of the repo and the readme contents.
    """
    contents = get_repo_contents(repo)
    readme_download_url = get_readme_download_url(contents)
    if readme_download_url == "":
        readme_contents = ""
    else:
        readme_contents = requests.get(readme_download_url).text
    return {
        "repo": repo,
        "language": get_repo_language(repo),
        "readme_contents": readme_contents,
    }


def scrape_github_data():
    """
    This function checks for a local csv file of raw data. If a local csv exists it reads it 
    to a df.
    
    If no local csv of raw data exists, it loops through all of the repos in the REPOS list above and
    and runs the process functions to scrape the repo name, language, and readme
    contents. It then writes the resulting dictionary of lists to a df, returning
    a dataframe of raw data. It then caches the raw data to a local csv file.
    """
    if os.path.isfile('raw_data.csv'):
        df = pd.read_csv('raw_data.csv')

    else:
        df = pd.DataFrame([process_repo(repo) for repo in REPOS])
        df.to_csv('raw_data.csv')

    return df


if __name__ == "__main__":
    data = scrape_github_data()
    json.dump(data, open("data.json", "w"), indent=1)